Loading data

In [86]:
import pandas as pd
import numpy as np
from scipy import stats

df_inception_old = pd.read_csv (r'C:\Users\20192236\Downloads\Good data\Good data\Inception_submission.csv')
df_efficient_old = pd.read_csv (r'C:\Users\20192236\Downloads\Good data\Good data\EfficientNet_submission.csv')
df_resnet_old = pd.read_csv (r'C:\Users\20192236\Downloads\Good data\Good data\ResNet_submission.csv')
df_Mobile_old = pd.read_csv (r'C:\Users\20192236\Downloads\Good data\Good data\MobileNetV2_submission.csv')
df_xception_old = pd.read_csv (r'C:\Users\20192236\Downloads\Good data\Good data\Xception_submission.csv')

df_inception = df_inception_old.rename(columns={'label':'label_inception'}, errors='raise')
df_efficient = df_efficient_old.rename(columns={'label':'label_efficient'}, errors='raise')
df_resnet = df_resnet_old.rename(columns={'label':'label_resnet'}, errors='raise')
df_mobile = df_vgg16_old.rename(columns={'label':'label_mobile'}, errors='raise')
df_xception = df_xception_old.rename(columns={'label':'label_xception'}, errors='raise')

all_probs = pd.DataFrame(data=[df_inception['id'], df_inception['label_inception'], df_xception['label_xception'], df_efficient['label_efficient'], df_resnet['label_resnet'], df_mobile['label_mobile']])
all_probs = all_probs.transpose()

Making predictions

In [87]:
mu = []
sigma = []
for i in range(len(all_probs)):
    mu.append(np.mean(all_probs.iloc[i, 1:6].values))
    sigma.append(np.std(all_probs.iloc[i, 1:6].values))

In [88]:
def predict_Bayes_class(X,mu_list,sigma_list): 
    #Returns the predicted class from an optimal bayes classifier - distributions must be known
    scores_list = []
    classes = len(mu_list)
    
    for p in range(classes):
        score = stats.multivariate_normal.pdf(X, mean=mu_list[p], cov=sigma_list[p])
        scores_list.append(score)
             
    return np.argmax(scores_list)

In [89]:
pred = []
for i in range(len(all_probs)):
        pred.append(predict_Bayes_class(all_probs.iloc[i, 1:6].values, [mu[i]], [sigma[i]]))

In [90]:
proba_list = []
proba_list_round = []
for i in range(len(pred)):
    if pred[i] == 0:
        proba_list.append(all_probs.iloc[i, 1])
    elif pred[i] == 1:
        proba_list.append(all_probs.iloc[i, 2])
    elif pred[i] == 2:
        proba_list.append(all_probs.iloc[i, 3])
    elif pred[i] == 3:
        proba_list.append(all_probs.iloc[i, 4])
    elif pred[i] == 4:
        proba_list.append(all_probs.iloc[i, 5])
    proba_list_round.append(int(round(proba_list[i], 0)))
proba_list

df = pd.DataFrame(proba_list)
df = df.set_index(all_probs['id'])
df = df.rename(columns={0: "label"})

df.to_csv(r"C:\Users\20192236\Documents\Project_Imaging\BOC.csv")

The second difficulty in BOC is choosing the prior probability for each hypothesis p(hi). For simplicity, the prior is usually set to be the uniform distribution [30].